In [ ]:
import os

path = "/kaggle/input/dig-4-bio-raman-transfer-learning-challenge"
files = os.listdir(path)
[(i, files[i]) for i in range(len(files))]

In [ ]:
import pandas as pd

df = pd.read_csv(os.path.join(path, files[5]))
df

In [ ]:
input_cols = df.columns[1:2049]
target_cols = df.columns[2050:]

In [ ]:
targets  = df[target_cols].dropna().to_numpy()
targets[:5]

In [ ]:
df = df[input_cols]
df['Unnamed: 1'] = df['Unnamed: 1'].str.replace('[\[\]]', '', regex=True).astype('int64')
df['Unnamed: 2048'] = df['Unnamed: 2048'].str.replace('[\[\]]', '', regex=True).astype('int64')

In [ ]:
import numpy as np

inputs = []
for i in range(0, len(df), 2):
    row1 = df.iloc[i].to_numpy()
    row2 = df.iloc[i+1].to_numpy()
    i = np.concatenate([row1, row2])
    inputs.append(i)
    
inputs = np.stack(inputs)
inputs.shape

In [ ]:
from sklearn.model_selection import train_test_split


train_inputs, eval_inputs, train_targets, eval_targets = train_test_split(
        inputs,
        targets,                      
        test_size=0.2,
        random_state=1000,
        shuffle=True
)

train_inputs.shape, eval_inputs.shape, train_targets.shape, eval_targets.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_inputs = scaler.fit_transform(train_inputs)
eval_inputs = scaler.transform(eval_inputs)

In [ ]:
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score


xgb = XGBRegressor(
    n_estimators=250,
    max_depth=2,
    learning_rate=0.05,
    objective='reg:squarederror',
    random_state=1000,
    eval_metric='rmse',
    reg_lambda=0.01,
)
xgb = MultiOutputRegressor(xgb)

xgb.fit(train_inputs, train_targets, verbose=True)
preds = xgb.predict(train_inputs)
r2_score(train_targets, preds, multioutput='uniform_average')

In [ ]:
preds = xgb.predict(eval_inputs)
r2_score(eval_targets, preds, multioutput='uniform_average')

In [ ]:
0.6156981749990731, 2, 0.01, 
0.6158108300334649, 2, 0.01, 250

In [ ]:
#def load_test():
test = pd.read_csv(os.path.join(path, files[6]))

row1 = test.columns[1:].to_numpy().copy()
row1[-1] = "5611"
row1 = row1.astype(np.float64)


cols = test.columns[1:]
test = test[cols]
test[" 5611]"] = test[" 5611]"].str.replace('[\[\]]', '', regex=True).astype('int64')
test = test.to_numpy()

test = np.insert(test, 0, row1, axis=0)
test = test.reshape(-1, 2, 2048).reshape(96, 2*2048)
test = test.astype(np.float32)
test.shape, test.dtype

In [ ]:
#def scale():
test = scaler.transform(test)
preds = xgb.predict(test)
preds.shape

In [ ]:
#def prepare_test():
column_names = ['Glucose', 'Sodium Acetate', 'Magnesium Sulfate']
preds_df = pd.DataFrame(preds, columns=column_names)
preds_df.insert(0, 'ID', [i+1 for i in range(len(preds_df))])
preds_df

In [ ]:
#def save_test():
preds_df.to_csv("baseline.csv", index=False)
f = pd.read_csv("/kaggle/working/xgb_baseline.csv")
f